In [1]:
# 필요한 패키지 임포트
import pandas as pd
import numpy as np
import tensorflow as tf

from transformers import TFBertForSequenceClassification, TFElectraForSequenceClassification, AutoTokenizer, ElectraTokenizer

from collections import Counter

In [2]:
# 경로 설정
klue_bert_path = "/content/drive/MyDrive/modu/DLTON/model/final_klue-bert_model_2412081140"
koelectra_path = "/content/drive/MyDrive/modu/DLTON/model/final_koelectra_model_2412082210"

In [3]:
TEST_FILE_PATH = "/content/drive/MyDrive/modu/DLTON/aiffel-dl-thon-online-10/test_cleansed_241208.csv"
OUTPUT_FILE = "/content/drive/MyDrive/modu/DLTON/submission_softvoting_2412091750.csv"
OUTPUT_CSV_FILE ="/content/drive/MyDrive/modu/DLTON/test_with_predictions_softvoting_2412091750.csv"
MAX_LENGTH = 489  # 토큰화된 입력의 최대 길이

In [4]:
# 클래스 매핑 정의
class_mapping = {
    "협박 대화": 0,
    "갈취 대화": 1,
    "직장 내 괴롭힘 대화": 2,
    "기타 괴롭힘 대화": 3,
    "일반 대화": 4
}

# class_mapping에서 클래스 이름 리스트 생성
class_labels = [k for k, v in sorted(class_mapping.items(), key=lambda item: item[1])]

# 클래스 수 계산
num_classes = len(class_mapping)

print("Class Labels:", class_labels)
print("Number of Classes:", num_classes)

Class Labels: ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']
Number of Classes: 5


In [5]:
# 모델과 토크나이저 로드 함수
def load_kluebert_model_and_tokenizer(model_path):
    """
    KLUE-BERT 모델과 토크나이저를 로드합니다.
    Args:
    - model_path (str): KLUE-BERT 모델 저장 경로
    Returns:
    - model: TFBertForSequenceClassification 모델
    - tokenizer: AutoTokenizer
    """
    model = TFBertForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

In [6]:
def load_koelectra_model_and_tokenizer(model_path):
    """
    KoELECTRA 모델과 토크나이저를 로드합니다.
    Args:
    - model_path (str): KoELECTRA 모델 저장 경로
    Returns:
    - model: TFElectraForSequenceClassification 모델
    - tokenizer: ElectraTokenizer
    """
    model = TFElectraForSequenceClassification.from_pretrained(model_path)
    tokenizer = ElectraTokenizer.from_pretrained(model_path)
    return model, tokenizer

In [7]:
# Soft Voting 앙상블 함수
def soft_voting_ensemble(models, tokenizers, texts, max_length=128):
    """
    Soft Voting을 이용해 여러 모델의 예측을 결합하여 최종 클래스를 예측합니다.
    Args:
    - models (list): 학습된 TensorFlow 모델 리스트.
    - tokenizers (list): 각 모델에 대응하는 Hugging Face 토크나이저 리스트.
    - texts (list): 예측할 텍스트 리스트.
    - max_length (int): 토큰화된 입력의 최대 길이.

    Returns:
    - final_predictions (list): Soft Voting으로 앙상블된 최종 예측 클래스 인덱스.
    - final_labels (list): Soft Voting으로 앙상블된 최종 예측 클래스 이름.
    """
    probabilities = []
    for model, tokenizer in zip(models, tokenizers):
        inputs = tokenizer(
            texts,
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="tf"
        )
        preds = model.predict(dict(inputs)).logits
        probabilities.append(tf.nn.softmax(preds, axis=1).numpy())

    avg_probabilities = np.mean(probabilities, axis=0)
    final_predictions = np.argmax(avg_probabilities, axis=1)
    final_labels = [class_labels[pred] for pred in final_predictions]

    return final_predictions, final_labels

In [8]:
# Prediction 결과 저장 함수
def predict_and_save_files(models, tokenizers, test_data, predictions_file, submission_file, max_length=128):
    """
    Soft Voting을 사용해 테스트 데이터를 예측하고, 예측 결과를 포함한 두 개의 파일을 저장합니다.
    Args:
    - models: 학습된 모델 리스트
    - tokenizers: Hugging Face 토크나이저 리스트
    - test_data: pandas DataFrame (idx, conversation 열 포함)
    - predictions_file (str): train_dataset_with_predictions.csv 저장 경로
    - submission_file (str): submission.csv 저장 경로
    - max_length (int): 토큰화된 입력의 최대 길이
    """
    texts = test_data['text'].tolist()

    # Soft Voting 앙상블 예측
    predicted_classes, predicted_labels = soft_voting_ensemble(models, tokenizers, texts, max_length)

    # 예측 결과를 데이터프레임에 추가
    test_data['predicted_class_index'] = predicted_classes
    test_data['predicted_class_name'] = predicted_labels

    # 클래스별 예측 개수 출력
    class_counts = Counter(predicted_labels)
    print("\nClass Prediction Counts:")
    for class_name, count in class_counts.items():
        print(f"{class_name}: {count}")

    # train_dataset_with_predictions.csv 저장
    test_data.to_csv(predictions_file, index=False)
    print(f"\nPredictions saved to {predictions_file}")

    # submission.csv 저장
    submission = pd.DataFrame({
        "idx": test_data['idx'],      # 파일 이름 또는 고유 식별자
        "target": predicted_classes   # 정수형 클래스 ID
    })
    submission.to_csv(submission_file, index=False)
    print(f"Submission file saved to {submission_file}")

In [9]:
# 모델 및 토크나이저 로드
klue_bert_model, klue_bert_tokenizer = load_kluebert_model_and_tokenizer(klue_bert_path)
koelectra_model, koelectra_tokenizer = load_koelectra_model_and_tokenizer(koelectra_path)

# 모델과 토크나이저 리스트
models = [klue_bert_model, koelectra_model]
tokenizers = [klue_bert_tokenizer, koelectra_tokenizer]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/modu/DLTON/model/final_klue-bert_model_2412081140.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFElectraForSequenceClassification.

All the layers of TFElectraForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/modu/DLTON/model/final_koelectra_model_2412082210.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraForSequenceClassification for predictions without further training.


In [10]:
# # 테스트 데이터
# texts = [
#     "이 문장은 테스트 문장입니다.",
#     "예제 문장이며 결과를 확인해 봅니다.",
#     "어떤 클래스에 속할지 궁금합니다."
# ]

# # Soft Voting 앙상블 예측
# ensemble_predictions = soft_voting_ensemble(models, tokenizers, texts)

# # 결과 출력
# print("Soft Voting Predictions:", ensemble_predictions)

# # 클래스 이름 매핑 (필요 시)
# class_labels = ["협박 대화", "갈취 대화", "직장 내 괴롭힘 대화", "기타 괴롭힘 대화", "일반 대화"]
# print("Predicted Classes:", [class_labels[pred] for pred in ensemble_predictions])

In [11]:
# 데이터 로드
test_data = pd.read_csv(TEST_FILE_PATH)

# 예측 및 저장
predict_and_save_files(
    models=models,
    tokenizers=tokenizers,
    test_data=test_data,
    predictions_file=OUTPUT_CSV_FILE,
    submission_file=OUTPUT_FILE,
    max_length=128
)

16/16 [==============================] - 259s 16s/step

Class Prediction Counts:
갈취 대화: 92
직장 내 괴롭힘 대화: 98
일반 대화: 81
기타 괴롭힘 대화: 148
협박 대화: 81

Predictions saved to /content/drive/MyDrive/modu/DLTON/test_with_predictions_softvoting_2412091750.csv
Submission file saved to /content/drive/MyDrive/modu/DLTON/submission_softvoting_2412091750.csv
